### Helena function for comparing MC true blob info to reco blob info


this is the function for assigning blob info to MC

In [2]:
import IC.invisible_cities.io.mcinfo_io as mcio


ModuleNotFoundError: No module named 'IC'

In [1]:
# 
def assign_trueblobs(true_0nubb_primary, tracks_0nubb_ROIovlp1trackfid, tag):
    diff_b1_x, diff_b1_y, diff_b1_z = [], [], []
    diff_b2_x, diff_b2_y, diff_b2_z = [], [], []

    totalevents, events12AB, events12BA, eventsNA = 0, 0, 0, 0
    
    if (tag == 'signal'):
        true_0nubb_primary_part1 = true_0nubb_primary[true_0nubb_primary.particle_id == 1]
        true_0nubb_primary_part2 = true_0nubb_primary[true_0nubb_primary.particle_id == 2]
        
    events = tracks_0nubb_ROIovlp1trackfid.event.unique()
    for nevt in events:
        #print(nevt)
        if nevt not in tracks_0nubb_ROIovlp1trackfid.event.unique():
            continue
        #if eventmap[eventmap.evt_number == nevt].index[0] not in true_0nubb_primary_part1.event_id.unique():
        #    continue
        if (tag == 'signal'):
            true_x_pA = float(true_0nubb_primary_part1[true_0nubb_primary_part1.event_id == nevt].final_x.values[0])
            true_y_pA = float(true_0nubb_primary_part1[true_0nubb_primary_part1.event_id == nevt].final_y.values[0])
            true_z_pA = float(true_0nubb_primary_part1[true_0nubb_primary_part1.event_id == nevt].final_z.values[0])

            true_x_pB = float(true_0nubb_primary_part2[true_0nubb_primary_part2.event_id == nevt].final_x.values[0])
            true_y_pB = float(true_0nubb_primary_part2[true_0nubb_primary_part2.event_id == nevt].final_y.values[0])
            true_z_pB = float(true_0nubb_primary_part2[true_0nubb_primary_part2.event_id == nevt].final_z.values[0])

            true_r_p1 = math.sqrt(true_x_pA*true_x_pA+true_y_pA*true_y_pA)
            true_r_p2 = math.sqrt(true_x_pB*true_x_pB+true_y_pB*true_y_pB)

        elif (tag == 'bkg'):
            true_x_pA = float(true_0nubb_primary[true_0nubb_primary.event_id == nevt].initial_x.values[0])
            true_y_pA = float(true_0nubb_primary[true_0nubb_primary.event_id == nevt].initial_y.values[0])
            true_z_pA = float(true_0nubb_primary[true_0nubb_primary.event_id == nevt].initial_z.values[0])

            true_x_pB = float(true_0nubb_primary[true_0nubb_primary.event_id == nevt].final_x.values[0])
            true_y_pB = float(true_0nubb_primary[true_0nubb_primary.event_id == nevt].final_y.values[0])
            true_z_pB = float(true_0nubb_primary[true_0nubb_primary.event_id == nevt].final_z.values[0])
            
        reco_x_blob1 = float(tracks_0nubb_ROIovlp1trackfid[tracks_0nubb_ROIovlp1trackfid.event == nevt].blob1_x.values[0])
        reco_y_blob1 = float(tracks_0nubb_ROIovlp1trackfid[tracks_0nubb_ROIovlp1trackfid.event == nevt].blob1_y.values[0])
        reco_z_blob1 = float(tracks_0nubb_ROIovlp1trackfid[tracks_0nubb_ROIovlp1trackfid.event == nevt].blob1_z.values[0])

        reco_x_blob2 = float(tracks_0nubb_ROIovlp1trackfid[tracks_0nubb_ROIovlp1trackfid.event == nevt].blob2_x.values[0])
        reco_y_blob2 = float(tracks_0nubb_ROIovlp1trackfid[tracks_0nubb_ROIovlp1trackfid.event == nevt].blob2_y.values[0])
        reco_z_blob2 = float(tracks_0nubb_ROIovlp1trackfid[tracks_0nubb_ROIovlp1trackfid.event == nevt].blob2_z.values[0]) 


        d_A1 = math.sqrt((true_x_pA-reco_x_blob1)**2+(true_y_pA-reco_y_blob1)**2+(true_z_pA-reco_z_blob1)**2)
        d_B1 = math.sqrt((true_x_pA-reco_x_blob2)**2+(true_y_pA-reco_y_blob2)**2+(true_z_pA-reco_z_blob2)**2)
        d_A2 = math.sqrt((true_x_pB-reco_x_blob1)**2+(true_y_pB-reco_y_blob1)**2+(true_z_pB-reco_z_blob1)**2)
        d_B2 = math.sqrt((true_x_pB-reco_x_blob2)**2+(true_y_pB-reco_y_blob2)**2+(true_z_pB-reco_z_blob2)**2)

        totalevents = totalevents+1
        if (d_A1<d_B1 and  d_A1<d_A2) or (d_B2<d_B1 and d_B2<d_A2):
            events12AB = events12AB+1
            diff_b1_x.append(reco_x_blob1 - true_x_pA)
            diff_b1_y.append(reco_y_blob1 - true_y_pA)

            diff_b2_x.append(reco_x_blob2 - true_x_pB)
            diff_b2_y.append(reco_y_blob2 - true_y_pB)

        elif (d_A2<d_B1 and d_A2<d_A1) or (d_B1<d_B2 and d_B1<d_A1):
            events12BA = events12BA+1
            diff_b1_x.append(reco_x_blob1 - true_x_pB)
            diff_b1_y.append(reco_y_blob1 - true_y_pB)

            diff_b2_x.append(reco_x_blob2 - true_x_pA)
            diff_b2_y.append(reco_y_blob2 - true_y_pA)

        else:
            eventsNA = eventsNA+1

    print(f'Total events = {totalevents}')
    print(f'Events 12 = {events12AB} ({100*events12AB/totalevents}%)')
    print(f'Events 21 = {events12BA} ({100*events12BA/totalevents}%)')
    print(f'Events NA = {eventsNA} ({100*eventsNA/totalevents}%)')
    
    return diff_b1_x, diff_b1_y, diff_b2_x, diff_b1_y

then you need to take `particles` from true info:

**this part should be different due to input, separate signal from background with Tl208 here**

In [ ]:
#0nubb files
particles_list = []
eventmap_list = []
#loop over all the files, to read 'Tracks' information
for nfile in range(1,200): 
    #print(nfile)
    filename = filepath_0nubb_tracks+f'{dataforma}.bb0nu.{tp_config}.{nfile}.test_max_hits.isaura.R24mm.h5'

    try:
        readfile_particles = mcio.load_mcparticles_df(filename).reset_index()
        readfile_eventmap = mcio.load_eventnumbermap(filename).set_index('nexus_evt')
    except:
        print('File {} not good'.format(filename))
        continue

    #readfile_particles.event_id = readfile_particles.event_id.map(evtmap.evt_number)
    particles_list.append(readfile_particles)
    eventmap_list.append(readfile_eventmap)

#'concat' command means: 'concatenate pandas objects along a particular 
# axis with optional set logic along the other axes.' -> following line 
# is needed to keep an order between the files

particles = pd.concat([dt for dt in particles_list]) 
eventmap = pd.concat([dt for dt in eventmap_list]) 
particles.columns  
#these are all the elements inside the file:

loop to do it :up:

In [ ]:
particles.event_id = particles.event_id.map(eventmap.evt_number)
primary = particles[particles.primary == True]
diff_b1_x, diff_b1_y, diff_b2_x, diff_b2_y = assign_trueblobs(primary, tracks_0nubb_ovlp1trackfid, 'signal')

plot

In [ ]:
plt.hist(diff_b1_x, bins = 50, color='r', histtype='step', label = 'x blob1')
plt.hist(diff_b1_y, bins = 50, color='r', histtype='step', label = 'y blob1', linestyle='dashed')

plt.xlabel('reco[tracks] - true[particle] (mm)')
plt.ylabel('events (a.u.)')
plt.title('0nubb')
plt.legend(loc=1)
plt.show()
